In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import cv2
import os
import numpy as np

import __init_path__
import env

from slam.models.relocalization.bovw import BoVW
from slam.utils import resize_image


font = {'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

dataset_root = Path('/datasets/odometry/KITTI_test/00/reloc')
difference_threshold = 100
matches_threshold = 40

In [ ]:
def read_image(path):
    image = cv2.imread(path.as_posix())
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, channels_num = image.shape
    small_height = height // 4
    small_width = width // 4
    image = resize_image(image, (small_width, small_height))
    return image

def visualize_image(df):
    if len(df) == 0:
        return
    
    frame_total = df.to_index.iloc[-1]
    frame_counter = 1
    
    fig, axes = plt.subplots(len(df) + 1, figsize=(20, 10  * len(df)))
    axes[0].title.set_text(f'to {df.iloc[0].to_index}')
    image = read_image(dataset_root.parent/'rgb'/f'{int(df.iloc[0].to_index)}.png')
    axes[0].imshow(image / 255)
            
    for index, row in df.iterrows():        
        image = read_image(dataset_root.parent/'rgb'/f'{int(row.to_index)}.png')
        axes[index + 1].title.set_text(f'from {row.from_index}. Matches {row.matches_num}')
        axes[index + 1].imshow(image / 255)
    plt.show()

def visualize_trajectory(df, difference_threshold, matches_threshold):
    matches = list(dataset_root.glob('*.csv'))
    matches.sort()
    for m in matches:
        df = pd.read_csv(m.as_posix())
        is_keyframe = (df.to_index - df.from_index) > difference_threshold
        df = df[is_keyframe].reset_index(drop=True)

        if len(df) == 0:
            continue

        df = df[df.matches_num >= matches_threshold].reset_index(drop=True)

        if len(df) > 0:
            print(df)
            visualize_image(df)    

In [ ]:
visualize_trajectory(df, difference_threshold=difference_threshold, matches_threshold=matches_threshold)